In [3]:
!pip install evaluate

In [4]:
import evaluate
import torch
import os
import csv
import os
import pandas as pd
import numpy as np
from torch import nn
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

GPU is available


In [6]:
accuracy = evaluate.load("accuracy")

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

os.environ["TOKENIZERS_PARALLELISM"] = "true"

from google.colab import drive
drive.mount('/content/drive')

file_path='/content/drive/MyDrive/Colab Notebooks/comp4211 project/shuffled_A_B_relation_train.csv'
with open(file_path, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    csv_table = list(reader)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
csv_table[1]

{'EventA': 'PersonX drops off the radar',
 'EventB': "['has peace an quiet', 'gets more sleep']",
 'relation': 'xEffect'}

In [8]:
id2label = {0: "oEffect", 1: "oReact", 2: "oWant", 3: "xAttr",
            4: "xEffect", 5: "xIntent", 6: "xNeed", 7: "xReact", 8: "xWant"}
label2id = {"oEffect": 0, "oReact": 1, "oWant": 2,  "xAttr": 3,
            "xEffect": 4, "xIntent": 5, "xNeed": 6, "xReact": 7, "xWant": 8}

before_table = []
for row in csv_table:
    before_row = {
        'text': (row['EventA'] + "==" + row['EventB']),
        'label': (label2id[row['relation']])
    }
    before_table.append(before_row)

In [9]:
df = pd.read_csv(file_path)
df.loc[:, 'label'] = 0
df['EventB'] = df['EventB'].str.replace("[", "", regex=False).str.replace("]", "", regex=False).str.replace("'", "", regex=False).str.strip()
df['label'] = df['relation'].map(label2id)
df.head(2)

,EventA,EventB,relation,label
0,PersonX causes trouble,"Is arrested, Thrown in jail",xEffect,4
1,PersonX drops off the radar,"has peace an quiet, gets more sleep",xEffect,4


In [10]:
df.head(10)

,EventA,EventB,relation,label
0,PersonX causes trouble,"Is arrested, Thrown in jail",xEffect,4
1,PersonX drops off the radar,"has peace an quiet, gets more sleep",xEffect,4
2,PersonX starts the fire,"to light a match, to pour gasoline everywhere",xNeed,6
3,PersonX attends to PersonY's duties,"to hear thanks, keep helping person",xWant,8
4,PersonX lets PersonY go,"sad, but helpful",xReact,7
5,PersonX is seriously hurt,"hopeless, weak",xAttr,3
6,PersonX buys a new laptop,"in need, capable",xAttr,3
7,PersonX lives ___ to the fullest,to have fun,xIntent,5
8,PersonX is nowhere to be found,worried,oReact,1
9,PersonX brings ___ to a crisis,to be safe,xIntent,5


In [11]:
text1 = []
label1 = []
text2 = []
label2 = []
text3 = []
label3 = []

index1 = 0
index2 = 0

prefix = "Relations include 0: oEffect, 1: oReact, 2: oWant, 3: xAttr, 4: xEffect, 5: xIntent, 6: xNeed, 7: xReact, 8: xWant.The illocutionary relation between the two sentences is [mask]."
for index, row in df.iterrows():
    if index < 16000:
        text1.append('[CLS]' + prefix + '[SEP]' + row['EventA'] + '[SEP]' + row['EventB'])
        label1.append(row['label'])

    elif index>=16000 and index<20000:
        text2.append('[CLS]' + prefix + '[SEP]' + row['EventA'] + '[SEP]' + row['EventB'])
        label2.append(row['label'])
    else:
      break

dataset1 = Dataset.from_dict({"label": label1, "text": text1})
dataset2 = Dataset.from_dict({"label": label2, "text": text2})
dataset3 = Dataset.from_dict({"label": label3, "text": text3})

dataset_dict = DatasetDict()
notuse_dataset_dict = DatasetDict()

dataset_dict["train"] = dataset1
dataset_dict["test"] = dataset2
notuse_dataset_dict["try"] = dataset3

In [12]:
num_elements_dataset1 = len(dataset1)
num_elements_dataset2 = len(dataset2)
num_elements_dataset3 = len(dataset3)

print(f"Dataset1元素数量: {num_elements_dataset1}")
print(f"Dataset2元素数量: {num_elements_dataset2}")
print(f"Dataset3元素数量: {num_elements_dataset3}")

# 查看前5个元素
print(dataset1[:5])
print(dataset2[:5])
print(dataset3[:5])

Dataset1元素数量: 16000
Dataset2元素数量: 4000
Dataset3元素数量: 0
{'label': [4, 4, 6, 8, 7], 'text': ['[CLS]Relations include 0: oEffect, 1: oReact, 2: oWant, 3: xAttr, 4: xEffect, 5: xIntent, 6: xNeed, 7: xReact, 8: xWant.The illocutionary relation between the two sentences is [mask].[SEP]PersonX causes trouble[SEP]Is arrested, Thrown in jail', '[CLS]Relations include 0: oEffect, 1: oReact, 2: oWant, 3: xAttr, 4: xEffect, 5: xIntent, 6: xNeed, 7: xReact, 8: xWant.The illocutionary relation between the two sentences is [mask].[SEP]PersonX drops off the radar[SEP]has peace an quiet, gets more sleep', '[CLS]Relations include 0: oEffect, 1: oReact, 2: oWant, 3: xAttr, 4: xEffect, 5: xIntent, 6: xNeed, 7: xReact, 8: xWant.The illocutionary relation between the two sentences is [mask].[SEP]PersonX starts the fire[SEP]to light a match, to pour gasoline everywhere', "[CLS]Relations include 0: oEffect, 1: oReact, 2: oWant, 3: xAttr, 4: xEffect, 5: xIntent, 6: xNeed, 7: xReact, 8: xWant.The illocutionary 

In [13]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


tokenized = dataset_dict.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base", num_labels=9, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="task1_deberta(4-epochs)(1)",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-c053d4bba956>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
trainer.evaluate()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: arwenwu959 (argument-mining). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.129800,0.746246,0.725000


Epoch,Training Loss,Validation Loss,Accuracy
1,1.129800,0.746246,0.725000
2,0.635400,0.568150,0.796750


{'eval_loss': 0.5681504011154175,
 'eval_accuracy': 0.79675,
 'eval_runtime': 29.4585,
 'eval_samples_per_second': 135.784,
 'eval_steps_per_second': 4.243,
 'epoch': 2.0}

In [15]:
trainer.train()
trainer.evaluate()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,1.205100,0.737271,0.723500
2,0.661400,0.625588,0.774750
3,0.486700,0.544021,0.817500
4,0.384000,0.528673,0.819250


{'eval_loss': 0.5286731719970703,
 'eval_accuracy': 0.81925,
 'eval_runtime': 32.1433,
 'eval_samples_per_second': 124.443,
 'eval_steps_per_second': 3.889,
 'epoch': 4.0}

In [17]:
!zip -r 'task1_deberta(4-epochs)(1).zip' '/content/task1_deberta(4-epochs)(1)'


  adding: content/task1_deberta(4-epochs)(1)/ (stored 0%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/ (stored 0%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/tokenizer_config.json (deflated 74%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/rng_state.pth (deflated 25%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/optimizer.pt (deflated 54%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/model.safetensors (deflated 22%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/trainer_state.json (deflated 69%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/training_args.bin (deflated 51%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/spm.model (deflated 50%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/special_tokens_map.json (deflated 50%)
  adding: content/task1_deberta(4-epochs)(1)/checkpoint-2000/config.json (deflated 56%)
  adding: content/task1_deberta(4-